# Pipeline Stage 4
Two things happen on processing source files from ScienceBase:

1. Individual source records go into a message queue for validation and processing
2. Scientific names are extracted and placed onto a message queue for processing initially against ITIS

In the current model of this process, we are creating a single database table containing all of the summarized properties for each name submitted by a State/Territory. That gives us the necessary data to respond to queries for each State/Territory but also to pull out the National List names.

The body of the messages in the mq_source_records queue processed here contains the properties from source files with a little bit of processing metadata indicating exactly where the records came from (sciencebase_item_id, source_file_url, source_file_date) along with metadata infused from collection records (historic_list, itis_identifier). A byproduct of running this through the Sqlite process that I used for the local processing mode, using the feature from the sqlite_utils package to generate a hash_id from the record contents, is that we detect and reject duplicate records at this stage. There are a number of cases where the original source files from the states included duplicates of the same name record.

In [1]:
import pysgcn
sgcn = pysgcn.sgcn.Sgcn()

In [2]:
processable_item = sgcn.get_message("mq_source_records")
processable_item

{'id': 'e18630d5fe5fbf14e6be7ca55355bc9100ada7a2',
 'date_inserted': '2019-12-20T20:28:09.247288',
 'body': {'scientific name': 'Acipenser oxyrhinchus desotoi',
  'common name': 'Gulf Sturgeon',
  'taxonomic category': 'Fish',
  'state': 'Louisiana',
  'sciencebase_item_id': 'https://www.sciencebase.gov/catalog/item/5787cd0ae4b0d27deb3754f2',
  'record_processed': '2019-12-20T20:28:08.826103',
  'source_file_date': '2016-07-14T17:33:42.000Z',
  'source_file_url': 'https://www.sciencebase.gov/catalog/file/get/5787cd0ae4b0d27deb3754f2?f=__disk__b4%2Fae%2Fbf%2Fb4aebf82009a2aaadaa4d7b84fdcade7589c722b',
  'year': '2005',
  'clean_scientific_name': 'Acipenser oxyrhinchus desotoi',
  'historic_list': False,
  'itis_override_id': None,
  'sppin_key': 'Scientific Name:Acipenser oxyrhinchus desotoi'}}

In [3]:
%%time
nonunique_records = list()
while processable_item is not None:
    try:
        sgcn.process_sgcn_source_record(processable_item["body"])
    except Exception as e:
        nonunique_records.append(processable_item["body"])
        print(e)
    sgcn.delete_message("mq_source_records", processable_item["id"])
    processable_item = sgcn.get_message("mq_source_records")

UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
UNIQUE constraint failed: sgcn.id
CPU times: user 2min 21s, sys: 1min 45s, total: 4min 7s
Wall time: 6min 33s


In the process of inserting the records into a database with a unique constraint built by generating an identifier as the MD5 hash of the record contents, we discover some duplicates. These seem to track back to actual duplicate entries in the original submissions which could have come about for a variery of reasons. The important part is that we do not lose any species/state/year combinations in the submitted data. The following codeblocks show the duplicate values we turned up and then check to make sure that we ended up with a species/state/year combination in the final data pushed to our table.

In [4]:
nonunique_records

[{'scientific name': 'Glyphyalinia sp.',
  'common name': 'terrestrial snails',
  'taxonomic category': 'Gastropod',
  'sciencebase_item_id': 'https://www.sciencebase.gov/catalog/item/595a9a13e4b0d1f9f0528537',
  'record_processed': '2019-12-20T20:28:22.630481',
  'source_file_date': '2017-07-03T19:24:06.000Z',
  'source_file_url': 'https://www.sciencebase.gov/catalog/file/get/595a9a13e4b0d1f9f0528537?f=__disk__e5%2F8b%2F61%2Fe58b61cae166b1965b572e8748189199ad848491',
  'state': 'Tennessee',
  'year': '2015',
  'clean_scientific_name': 'Glyphyalinia',
  'historic_list': False,
  'itis_override_id': None,
  'sppin_key': 'Scientific Name:Glyphyalinia'},
 {'scientific name': 'Glyphyalinia sp.',
  'common name': 'terrestrial snails',
  'taxonomic category': 'Gastropod',
  'sciencebase_item_id': 'https://www.sciencebase.gov/catalog/item/595a9a13e4b0d1f9f0528537',
  'record_processed': '2019-12-20T20:28:22.630481',
  'source_file_date': '2017-07-03T19:24:06.000Z',
  'source_file_url': 'https

In [12]:
for record in nonunique_records:
    if f'{record["state"]}_{record["year"]}' not in [f'{r["state"]}_{r["year"]}' for r in sgcn.sql_data.get_select_records(
        "sgcn",
        "sgcn",
        "clean_scientific_name = ?",
        record["clean_scientific_name"]
    )]:
        print(record)
    

At this point in the process we have the start to a final SGCN table containing all records for all states, years, and submitted names in a schema that contains source properties and infused metadata tracking the ScienceBase provenance for each record. We can now proceed to build out additional properties for our simple, synthesized data model to include summaries of valid taxon matches from ITIS or WoRMS as well as other useful details on Federal listing status, conservation status from recognized sources (IUCN, NatureServe), and other summary properties that may prove useful in examining the SGCN species list overall.